In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1)
sns.set_palette("colorblind")

nvalloc = Path("module/220826-153744")
buddy = Path("module/220826-153925")


In [ ]:
nvalloc_d = pd.read_csv(nvalloc / "bulk" / "out.csv")
nvalloc_d["alloc"] = "NVAlloc"
buddy_d = pd.read_csv(buddy / "bulk" / "out.csv")

pgd = pd.concat([nvalloc_d, buddy_d], ignore_index=True)
pgd = pgd[pgd["iteration"] != 0]
pgd["cores"] = pgd["x"]
pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["cores", "alloc", "get", "put"]].melt(
    id_vars=["cores", "alloc"],
    value_vars=["get", "put"], value_name="time")
# pgd = pgd.groupby(["cores", "alloc", "variable", "mem"]).min()
order = ["Kernel", "NVAlloc"]
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)
g.set(xticks=[1, 16, 32, 48, 64])
g.set(ylim=(0, 2700))
g.legend.set_title("Allocator")
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{col_name}")


In [ ]:
nvalloc_d = pd.read_csv(nvalloc / "repeat" / "out.csv")
nvalloc_d["alloc"] = "NVAlloc"
buddy_d = pd.read_csv(buddy / "repeat" / "out.csv")

pgd = pd.concat([nvalloc_d, buddy_d], ignore_index=True)
pgd = pgd[pgd["iteration"] != 0]
pgd["cores"] = pgd["x"]

# data = data.groupby(["cores", "alloc", "mem"]).min()
order = ["Kernel", "NVAlloc"]
g = sns.relplot(data=pgd, x="x", y="get_avg", kind="line",
                style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)

g.set(xticks=[1, 16, 32, 48, 64])
g.set(ylim=(0, 400))
# g.set(xlim=(1, 17))
g.legend.set_title("Allocator")
g.set(xlabel="cores")
g.set(ylabel="time in ns")
g.set_titles("{col_name}")


In [ ]:
nvalloc_d = pd.read_csv(nvalloc / "rand" / "out.csv")
nvalloc_d["alloc"] = "NVAlloc"
buddy_d = pd.read_csv(buddy / "rand" / "out.csv")

pgd = pd.concat([nvalloc_d, buddy_d], ignore_index=True)
pgd["cores"] = pgd["x"]
pgd = pgd[pgd["iteration"] != 0]

# data = data.groupby(["cores", "alloc", "mem"]).min()
order = ["Kernel", "NVAlloc"]
g = sns.relplot(data=pgd, x="x", y="get_avg", kind="line",
                style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)

g.set(xticks=[1, 16, 32, 48, 64])
g.set(ylim=(0, 400))
# g.set(xlim=(1, 17))
g.legend.set_title("Allocator")
g.set(xlabel="cores")
g.set(ylabel="time in ns")
g.set_titles("{col_name}")
